In [ ]:
from plotly.data import gapminder
from dash import dcc, html, Dash, callback, Input, Output
import plotly.express as px
import plotly.graph_objects as go

In [61]:
# Sample data
df = px.data.gapminder()

# Initialize Dash app
app = dash.Dash(__name__)

# Layout
app.layout = html.Div([
    html.H1("Gapminder Dataset Analysis"),
    dcc.Dropdown(
        id="country-dropdown",
        options=[
            {"label": country, "value": country}
            for country in df["country"].unique()
        ],
        value="United States",  # Default value
        placeholder="Select a country",
    ),
    dcc.Graph(id="line-chart")
])

# Callback
@app.callback(
    Output("line-chart", "figure"),
    [Input("country-dropdown", "value")]
)
def update_chart(selected_country):
    filtered_df = df[df["country"] == selected_country]
    fig = px.line(
        filtered_df,
        x="year",
        y="gdpPercap",
        title=f"GDP Per Capita of {selected_country}"
    )
    return fig

# Run app
if __name__ == "__main__":
    app.run_server(debug=True)


In [58]:
from plotly.data import gapminder
gapminder_df = gapminder(datetimes=True, centroids=True, pretty_names=True)



In [59]:
gapminder_df["Year"] = gapminder_df.Year.dt.year

gapminder_df.head()

,Country,Continent,Year,Life Expectancy,Population,GDP per Capita,ISO Alpha Country Code,ISO Numeric Country Code,Centroid Longitude,Centroid Latitude
0,Afghanistan,Asia,1952,28.801,8425333,779.445314,AFG,4,65.0,33.0
1,Afghanistan,Asia,1957,30.332,9240934,820.853030,AFG,4,65.0,33.0
2,Afghanistan,Asia,1962,31.997,10267083,853.100710,AFG,4,65.0,33.0
3,Afghanistan,Asia,1967,34.020,11537966,836.197138,AFG,4,65.0,33.0
4,Afghanistan,Asia,1972,36.088,13079460,739.981106,AFG,4,65.0,33.0


In [62]:
from plotly.data import gapminder
from dash import dcc, html, Dash, callback, Input, Output
import plotly.express as px
import plotly.graph_objects as go

css = ["https://cdn.jsdelivr.net/npm/bootstrap@5.3.1/dist/css/bootstrap.min.css", ]
app = Dash(name="Gapminder Dashboard", external_stylesheets=css)

################### DATASET ####################################
gapminder_df = gapminder(datetimes=True, centroids=True, pretty_names=True)
gapminder_df["Year"] = gapminder_df.Year.dt.year

#################### CHARTS #####################################
def create_table():
    fig = go.Figure(data=[go.Table(
        header=dict(values=gapminder_df.columns, align='left'),
        cells=dict(values=gapminder_df.values.T, align='left'))
    ]
    )
    fig.update_layout(paper_bgcolor="#e5ecf6", margin={"t":0, "l":0, "r":0, "b":0}, height=700)
    return fig

def create_population_chart(continent="Asia", year=1952, ):
    filtered_df = gapminder_df[(gapminder_df.Continent==continent) & (gapminder_df.Year==year)]
    filtered_df = filtered_df.sort_values(by="Population", ascending=False).head(15)

    fig = px.bar(filtered_df, x="Country", y="Population", color="Country",
                   title="Country {} for {} Continent in {}".format("Population", continent, year),
                   text_auto=True)
    fig.update_layout(paper_bgcolor="#e5ecf6", height=600)
    return fig

def create_gdp_chart(continent="Asia", year=1952):
    filtered_df = gapminder_df[(gapminder_df.Continent==continent) & (gapminder_df.Year==year)]
    filtered_df = filtered_df.sort_values(by="GDP per Capita", ascending=False).head(15)

    fig = px.bar(filtered_df, x="Country", y="GDP per Capita", color="Country",
                   title="Country {} for {} Continent in {}".format("GDP per Capita", continent, year),
                   text_auto=True)
    fig.update_layout(paper_bgcolor="#e5ecf6", height=600)
    return fig

def create_life_exp_chart(continent="Asia", year=1952):
    filtered_df = gapminder_df[(gapminder_df.Continent==continent) & (gapminder_df.Year==year)]
    filtered_df = filtered_df.sort_values(by="Life Expectancy", ascending=False).head(15)

    fig = px.bar(filtered_df, x="Country", y="Life Expectancy", color="Country",
                   title="Country {} for {} Continent in {}".format("Life Expectancy", continent, year),
                   text_auto=True)
    fig.update_layout(paper_bgcolor="#e5ecf6", height=600)
    return fig

def create_choropleth_map(variable, year):
    filtered_df = gapminder_df[gapminder_df.Year==year]

    fig = px.choropleth(filtered_df, color=variable, 
                        locations="ISO Alpha Country Code", locationmode="ISO-3",
                        color_continuous_scale="RdYlBu", hover_data=["Country", variable],
                        title="{} Choropleth Map [{}]".format(variable, year)
                     )

    fig.update_layout(dragmode=False, paper_bgcolor="#e5ecf6", height=600, margin={"l":0, "r":0})
    return fig

##################### WIDGETS ####################################
continents = gapminder_df.Continent.unique()
years = gapminder_df.Year.unique()

cont_population = dcc.Dropdown(id="cont_pop", options=continents, value="Asia",clearable=False)
year_population = dcc.Dropdown(id="year_pop", options=years, value=1952,clearable=False)

cont_gdp = dcc.Dropdown(id="cont_gdp", options=continents, value="Asia",clearable=False)
year_gdp = dcc.Dropdown(id="year_gdp", options=years, value=1952,clearable=False)

cont_life_exp = dcc.Dropdown(id="cont_life_exp", options=continents, value="Asia",clearable=False)
year_life_exp = dcc.Dropdown(id="year_life_exp", options=years, value=1952,clearable=False)

year_map = dcc.Dropdown(id="year_map", options=years, value=1952,clearable=False)
var_map = dcc.Dropdown(id="var_map", options=["Population", "GDP per Capita", "Life Expectancy"],
                        value="Life Expectancy",clearable=False)

##################### APP LAYOUT ####################################
app.layout = html.Div([
    html.Div([
        html.H1("Gapminder Dataset Analysis", className="text-center fw-bold m-2"),
        html.Br(),
        dcc.Tabs([
            dcc.Tab([html.Br(),
                     dcc.Graph(id="dataset", figure=create_table())], label="Dataset"),
            dcc.Tab([html.Br(), "Continent", cont_population, "Year", year_population, html.Br(),
                     dcc.Graph(id="population")], label="Population"),
            dcc.Tab([html.Br(), "Continent", cont_gdp, "Year", year_gdp, html.Br(),
                     dcc.Graph(id="gdp")], label="GDP Per Capita"),
            dcc.Tab([html.Br(), "Continent", cont_life_exp, "Year", year_life_exp, html.Br(),
                     dcc.Graph(id="life_expectancy")], label="Life Expectancy"),
            dcc.Tab([html.Br(), "Variable", var_map, "Year", year_map, html.Br(),
                     dcc.Graph(id="choropleth_map")], label="Choropleth Map"),
        ])
    ], className="col-8 mx-auto"),
], style={"background-color": "#e5ecf6", "height": "100vh"})

##################### CALLBACKS ####################################
@callback(Output("population", "figure"), [Input("cont_pop", "value"), Input("year_pop", "value"),])
def update_population_chart(continent, year):
    return create_population_chart(continent, year)

@callback(Output("gdp", "figure"), [Input("cont_gdp", "value"), Input("year_gdp", "value"),])
def update_gdp_chart(continent, year):
    return create_gdp_chart(continent, year)

@callback(Output("life_expectancy", "figure"), [Input("cont_life_exp", "value"), Input("year_life_exp", "value"),])
def update_life_exp_chart(continent, year):
    return create_life_exp_chart(continent, year)

@callback(Output("choropleth_map", "figure"), [Input("var_map", "value"), Input("year_map", "value"),])
def update_map(var_map, year):
    return create_choropleth_map(var_map, year)

if __name__ == "__main__":
    app.run(debug=True)